# INSTALL AND IMPORT DEPENDECIES

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [3]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# Make Some Detections

In [4]:
import cv2
import mediapipe as mp

# Initialize MediaPipe holistic model and drawing utilities
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks using the correct face_mesh connections
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image, 
                results.face_landmarks, 
                mp.solutions.face_mesh.FACEMESH_CONTOURS,  
                mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
            )
        
        # 2. Right hand
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, 
                results.right_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS, 
                mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
            )

        # 3. Left Hand
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, 
                results.left_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS, 
                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
            )

        # 4. Pose Detections
        if results.pose_landmarks:  # Corrected the typo here
            mp_drawing.draw_landmarks(
                image, 
                results.pose_landmarks,  # Corrected the typo here
                mp_holistic.POSE_CONNECTIONS, 
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
                        
        cv2.imshow('Raw Webcam Feed', image)

        # Wait for key press
        key = cv2.waitKey(10)
        if key & 0xFF == ord('q'):
            print("Quitting...")
            break

cap.release()
cv2.destroyAllWindows()


Quitting...


In [5]:
results.face_landmarks.landmark[0].z

-0.029488002881407738

# Capture Landmarks & Export to CSV

In [6]:
import csv
import os
import numpy as np

In [7]:
len(results.pose_landmarks.landmark)

33

In [8]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [9]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [1]:
# landmarks

In [14]:
with open('coords8.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [21]:
class_name = "NAMASTE MAM"

In [22]:
import cv2
import mediapipe as mp
import numpy as np
import csv

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh  # Import face_mesh for FACEMESH_CONTOURS

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks using FACEMESH_CONTOURS
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image, 
                results.face_landmarks, 
                mp_face_mesh.FACEMESH_CONTOURS,  
                mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
            )
        
        # 2. Right hand
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, 
                results.right_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS, 
                mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
            )

        # 3. Left Hand
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, 
                results.left_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS, 
                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
            )

        # 4. Pose Detections
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, 
                results.pose_landmarks,  
                mp_holistic.POSE_CONNECTIONS, 
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
        
        # Export coordinates
        try:
            # Extract Pose landmarks
            if results.pose_landmarks:
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            else:
                pose_row = []

            # Extract Face landmarks
            if results.face_landmarks:
                face = results.face_landmarks.landmark
                face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            else:
                face_row = []
            
            # Concatenate rows if both pose and face landmarks are detected
            if pose_row or face_row:
                row = pose_row + face_row
                
                # Example class name (adjust as needed)
                row.insert(0, class_name)
                
                # Export to CSV
                with open('coords8.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(row) 
            
        except Exception as e:
            print(f"An error occurred: {e}")
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# Train Custom Model Using Scikit Learn


# Read in Collected Data and Process

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [24]:
df = pd.read_csv('coords8.csv')

In [25]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,HAPPY,0.513914,0.570174,-0.755887,0.999658,0.532811,0.489570,-0.727367,0.999192,0.551666,...,-0.004336,0.0,0.604620,0.439329,0.015744,0.0,0.609918,0.431322,0.016492,0.0
1,HAPPY,0.525606,0.554433,-1.717913,0.999691,0.552448,0.479689,-1.626044,0.999270,0.569988,...,-0.004074,0.0,0.603830,0.463457,0.015970,0.0,0.609375,0.457078,0.016726,0.0
2,HAPPY,0.538876,0.544821,-1.843600,0.999716,0.565626,0.471796,-1.759026,0.999331,0.582288,...,-0.000917,0.0,0.611185,0.469879,0.018974,0.0,0.616632,0.464530,0.019612,0.0
3,HAPPY,0.548343,0.544213,-1.831506,0.999723,0.574768,0.471701,-1.737521,0.999332,0.591490,...,0.000061,0.0,0.619994,0.471821,0.020716,0.0,0.625322,0.466696,0.021367,0.0
4,HAPPY,0.551609,0.545018,-1.754411,0.999722,0.578316,0.472249,-1.662068,0.999337,0.595110,...,-0.001327,0.0,0.622618,0.476898,0.018743,0.0,0.627927,0.471978,0.019370,0.0


In [26]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1274,NAMASTE MAM,0.463873,0.451814,-0.859854,0.999915,0.488827,0.372774,-0.807855,0.999860,0.509700,...,-0.004107,0.0,0.537989,0.385661,0.014946,0.0,0.544739,0.377034,0.015620,0.0
1275,NAMASTE MAM,0.467246,0.451624,-0.807738,0.999919,0.492614,0.372543,-0.752328,0.999865,0.514158,...,-0.000461,0.0,0.539822,0.389966,0.016779,0.0,0.547096,0.379761,0.017525,0.0
1276,NAMASTE MAM,0.475665,0.451706,-0.565355,0.999911,0.495906,0.374578,-0.511803,0.999847,0.514407,...,-0.003743,0.0,0.547790,0.392621,0.013869,0.0,0.554333,0.384929,0.014260,0.0
1277,NAMASTE MAM,0.475589,0.455343,-0.717044,0.999918,0.497487,0.376781,-0.647947,0.999857,0.515744,...,-0.000373,0.0,0.545306,0.391084,0.023099,0.0,0.551421,0.383382,0.024188,0.0
1278,NAMASTE MAM,0.475439,0.458170,-0.742995,0.999925,0.498790,0.378258,-0.684525,0.999869,0.517129,...,-0.002122,0.0,0.547546,0.387556,0.020377,0.0,0.553554,0.379692,0.021347,0.0


In [27]:
df[df['class']=='HAPPY']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,HAPPY,0.513914,0.570174,-0.755887,0.999658,0.532811,0.489570,-0.727367,0.999192,0.551666,...,-0.004336,0.0,0.604620,0.439329,0.015744,0.0,0.609918,0.431322,0.016492,0.0
1,HAPPY,0.525606,0.554433,-1.717913,0.999691,0.552448,0.479689,-1.626044,0.999270,0.569988,...,-0.004074,0.0,0.603830,0.463457,0.015970,0.0,0.609375,0.457078,0.016726,0.0
2,HAPPY,0.538876,0.544821,-1.843600,0.999716,0.565626,0.471796,-1.759026,0.999331,0.582288,...,-0.000917,0.0,0.611185,0.469879,0.018974,0.0,0.616632,0.464530,0.019612,0.0
3,HAPPY,0.548343,0.544213,-1.831506,0.999723,0.574768,0.471701,-1.737521,0.999332,0.591490,...,0.000061,0.0,0.619994,0.471821,0.020716,0.0,0.625322,0.466696,0.021367,0.0
4,HAPPY,0.551609,0.545018,-1.754411,0.999722,0.578316,0.472249,-1.662068,0.999337,0.595110,...,-0.001327,0.0,0.622618,0.476898,0.018743,0.0,0.627927,0.471978,0.019370,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,HAPPY,0.531190,0.518295,-1.511969,0.999868,0.563890,0.450110,-1.441520,0.999836,0.582709,...,-0.006787,0.0,0.599835,0.453450,0.009972,0.0,0.605537,0.447732,0.010365,0.0
209,HAPPY,0.530310,0.518089,-1.341371,0.999863,0.563074,0.449940,-1.272727,0.999821,0.582276,...,-0.006498,0.0,0.599645,0.452723,0.011696,0.0,0.605336,0.446644,0.012268,0.0
210,HAPPY,0.528273,0.518121,-1.336761,0.999868,0.561046,0.449234,-1.270611,0.999821,0.581126,...,-0.006337,0.0,0.600312,0.453434,0.012488,0.0,0.605955,0.447842,0.013025,0.0
211,HAPPY,0.527768,0.517869,-1.303383,0.999867,0.560882,0.446860,-1.238716,0.999805,0.581097,...,-0.005774,0.0,0.602541,0.454298,0.014016,0.0,0.608193,0.448347,0.014654,0.0


In [28]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [30]:
y_test

1074    NAMASTE MAM
1059    NAMASTE MAM
1125    NAMASTE MAM
864     NAMASTE MAM
1149    NAMASTE MAM
           ...     
1129    NAMASTE MAM
842     NAMASTE MAM
945     NAMASTE MAM
538        HURRAYYY
155           HAPPY
Name: class, Length: 384, dtype: object

# Train Machine Learning Classification Model

In [31]:
  from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [32]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [33]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Define pipelines with an imputer to handle missing values
pipelines = {
    'logistic': Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
        ('classifier', LogisticRegression())
    ]),
    # Add other pipelines similarly
}

fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model


C:\Users\Dell\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
 fit_models

{'logistic': Pipeline(steps=[('imputer', SimpleImputer()),
                 ('classifier', LogisticRegression())])}

In [2]:
fit_models['logistic'].predict(X_test)

In [36]:
print(fit_models.keys())


dict_keys(['logistic'])


# Evaluate and Serialize Model

In [37]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [38]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

logistic 0.9869791666666666


In [3]:
# fit_models['rf'].predict(X_test)
fit_models['logistic'].predict(X_test)

In [40]:
print(fit_models.keys())

dict_keys(['logistic'])


In [41]:
y_test

1074    NAMASTE MAM
1059    NAMASTE MAM
1125    NAMASTE MAM
864     NAMASTE MAM
1149    NAMASTE MAM
           ...     
1129    NAMASTE MAM
842     NAMASTE MAM
945     NAMASTE MAM
538        HURRAYYY
155           HAPPY
Name: class, Length: 384, dtype: object

In [42]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['logistic'], f)

# Make Detections with Model

In [43]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [44]:
model

Pipeline(steps=[('imputer', SimpleImputer()),
                ('classifier', LogisticRegression())])

In [4]:
    import cv2
    import mediapipe as mp
    import numpy as np
    import pandas as pd  # Import pandas for DataFrame
    import csv

    # Initialize MediaPipe holistic model and drawing utilities
    mp_holistic = mp.solutions.holistic
    mp_drawing = mp.solutions.drawing_utils
    mp_face_mesh = mp.solutions.face_mesh  # Import face_mesh for FACEMESH_CONTOURS

    # Assuming model is already trained and loaded, or add a placeholder
    # model = your_model_here

    cap = cv2.VideoCapture(0)

    # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor Feed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False        

            # Make Detections
            results = holistic.process(image)
            # print(results.face_landmarks)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # 1. Draw face landmarks using the correct FACE_CONNECTIONS from face_mesh
            mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_CONTOURS, 
                                     mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                     mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                     )

            # 2. Right hand
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                     )

            # 3. Left Hand
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                     )

            # 4. Pose Detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )
            # Export coordinates
            try:
                # Extract Pose landmarks
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                # Extract Face landmarks
                face = results.face_landmarks.landmark
                face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

                # Concatenate rows
                row = pose_row + face_row

                # Make Detections
                X = pd.DataFrame([row])
                body_language_class = model.predict(X)[0]  # Assuming model is loaded
                body_language_prob = model.predict_proba(X)[0]
                print(body_language_class, body_language_prob)

                # Grab ear coords
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [640,480]).astype(int))

                cv2.rectangle(image, 
                              (coords[0], coords[1]+5), 
                              (coords[0]+len(body_language_class)*20, coords[1]-30), 
                              (245, 117, 16), -1)
                cv2.putText(image, body_language_class, coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class.split(' ')[0]
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"An error occurred: {e}")

            cv2.imshow('Raw Webcam Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()


An error occurred: 'NoneType' object has no attribute 'landmark'
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not defined
An error occurred: name 'model' is not def

In [ ]:
 tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))